### Q1 - Running Elastic

In [1]:
%pip install requests

Note: you may need to restart the kernel to use updated packages.


In [2]:
import requests

response = requests.get("http://localhost:9200")
data = response.json()

data["version"]["build_hash"]

'dbcbbbd0bc4924cfeb28929dc05d82d662c527b7'

In [3]:
docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

### Q2 - Indexing Data

In [6]:
%pip install elasticsearch==8.17.0 tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 KB 2.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [5]:
from elasticsearch import Elasticsearch

es_client = Elasticsearch('http://localhost:9200')

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es_client.indices.create(
    index="course-questions", 
    body=index_settings
)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [7]:
from tqdm import tqdm

for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|██████████| 948/948 [00:05<00:00, 176.57it/s]


### Q3 - Searching

In [14]:
query = 'How do execute a command on a Kubernetes pod?'

search_query = {
    "size": 1,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            },
        }
    }
}

response = es_client.search(index=index_name, body=search_query)
response["hits"]["hits"][0]["_score"]

44.50556

### Q4 - Filtering

In [25]:
query = 'How do I copy a file to a Docker container?'

search_query = {
    "size": 3,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": "machine-learning-zoomcamp"
                }
            }
        },
    },
}

response = es_client.search(index=index_name, body=search_query)
response["hits"]["hits"][2]["_source"]["question"]

'How do I copy files from a different folder into docker container’s working directory?'

### Q5 - Building a prompt

In [26]:
result_docs = []
    
for hit in response['hits']['hits']:
    result_docs.append(hit['_source'])

In [36]:
context_template = """
Q: {question}
A: {text}
""".strip()

context = ""
for doc in result_docs:
    context += context_template.format(question=doc['question'], text=doc['text'])
    context += "\n\n"

In [40]:
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

prompt = prompt_template.format(question=query, context=context).strip()
len(prompt)

1448

### Q6 - Tokens

In [41]:
%pip install tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 5.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.7/781.7 KB 10.7 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [44]:
import tiktoken

encoding = tiktoken.encoding_for_model("gpt-4o")
tokens = encoding.encode(prompt)
len(tokens)

321

Decoding a single token as bytes:

In [49]:
encoding.decode_single_token_bytes(tokens[2])

b' course'

### Bonus: Generating LLM answer

In [51]:
%pip install openai python-dotenv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 730.3/730.3 KB 7.9 MB/s eta 0:00:00a 0:00:01
  Using cached jiter-0.10.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (352 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 444.8/444.8 KB 17.1 MB/s eta 0:00:00
  Using cached distro-1.9.0-py3-none-any.whl (20 kB)
  Using cached httpx-0.28.1-py3-none-any.whl (73 kB)
  Using cached sniffio-1.3.1-py3-none-any.whl (10 kB)
  Using cached anyio-4.9.0-py3-none-any.whl (100 kB)
  Using cached httpcore-1.0.9-py3-none-any.whl (78 kB)
  Using cached h11-0.16.0-py3-none-any.whl (37 kB)
  Using cached annotated_types-0.7.0-py3-none-any.whl (13 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 19.7 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [53]:
import os
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv()

client = OpenAI(
    base_url="https://api.groq.com/openai/v1",
    api_key=os.environ.get("GROQ_API_KEY")
)

response = client.chat.completions.create(
    model='llama-3.3-70b-versatile',
    messages=[{"role": "user", "content": prompt}]
)

answer = response.choices[0].message.content
answer

'To copy a file to a Docker container, you can use the `docker cp` command. The basic syntax is as follows: \n`docker cp /path/to/local/file container_id:/path/in/container` \n\nYou need to replace `/path/to/local/file` with the path to the file on your local machine, and `container_id` with the ID of the running Docker container. The file will be copied to the path specified by `/path/in/container` inside the container.'

Let's calculate the number of output tokens:

In [54]:
encoding = tiktoken.encoding_for_model("gpt-4o")
len(encoding.encode(answer))

97

### Bonus: Calculate costs

Supposing we are using OpenAI's gpt4o model and 1000 requests:  
- Total input tokens: `321 * 1000 = 321k`  
- Total output tokens: `97 * 1000 = 97k`  

Based on the pricing per 1K input/output tokens, we get:  
`321k * $0.005/1k + 97k * $0.015/1k = $3.06`.